In [ ]:
# 해당 파일은 python 3.10.9에서 작성 되었습니다.

In [ ]:
# pip install selenium

In [ ]:
# pip install --upgrade selenium

In [ ]:
# pip install --upgrade urllib3 chardet

# 이미지 크롤링

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from urllib.request import urlretrieve
import pandas as pd
import os
import time
import random

# 최소 및 최대 대기 시간 설정 (예: 'min_wait초 에서 'max_wait'초 사이의 랜덤 대기)
min_wait = 2 # 최소 대기 시간 (초)
max_wait = 4 # 최대 대기 시간 (초)

# 랜덤 대기 시간 생성
random_wait = random.uniform(min_wait, max_wait)

ind = 1 # 현재 복사한 순서
ind_ = 50 # 현재 페이지에 결번이 있으며 결번 뒤의 지정 번호 까지 반복하고 페이지를 넘어감
no = 0  # 페이지 번호
list_1 = [] # 결과물이 저장되는 리스트

# 찾을 검색어
find = '전동킥보드'

driver = webdriver.Chrome()
driver.get(f"https://search.shopping.naver.com/search/all?where=all&frm=NVSCTAB&query={find}")

time.sleep(5) # 브라우저가 열릴떄 까지 대기

# 이미지 저장할 폴더 생성
save_folder = os.path.join(os.getcwd(), "data")
os.makedirs(save_folder, exist_ok=True)

# 반복할 횟수 설정
repeat_limit = 3

while no <= repeat_limit:
    try:
        img_element = driver.find_element(by = 'xpath', value = f'//*[@id="content"]/div[1]/div[2]/div/div[{ind}]/div/div/div[1]/div/a/img')
        img_src = img_element.get_attribute("src")
        img_alt = img_element.get_attribute("alt")
        list_1 = [img_src, img_alt]
        
        try:
            file_name = f"{no + 1}-{ind}-{list_1[1]}.jpg"
            file_path = os.path.join(save_folder, file_name)
            urlretrieve(list_1[0], file_path)
            print(f"다운로드 완료: {file_name}")
            
        except Exception as e:
            print(f"다운로드 중 오류 발생: {e}")
            
        time.sleep(random_wait) # import time
        ind += 1

    except:
        if ind < ind_: 
            # 스크롤 내리기 (페이지 다운 키를 사용)
            driver.find_element(by = By.TAG_NAME, value = "body").send_keys(Keys.PAGE_DOWN)
            ind += 1
            continue
        
        else:
            # 다음 페이지로 넘어가기
            try:
                no += 1
                driver.find_element(by = 'xpath', value = f'//*[@id="page_area"]/a[{no}]').click()
                print(f"{no} 페이지의 다운로드를 완료하였습니다.")
                ind = 1

                time.sleep(random_wait) # import time

            except:
                break


# 택스트 크롤링

In [ ]:
# 패키지 불러오기
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import time
import random

# 최소 및 최대 대기 시간 설정 (예: 'min_wait초 에서 'max_wait'초 사이의 랜덤 대기)
min_wait = 1  # 최소 대기 시간 (초)
max_wait = 2  # 최대 대기 시간 (초)

# 랜덤 대기 시간 생성
random_wait = random.uniform(min_wait, max_wait)

ind = 1 # 현재 복사한 순서
no = 0  # 지역목록 순서
list_1 = [] # 결과물이 저장되는 리스트

# 찾을 검색어
find = '전동킥보드'

driver = webdriver.Chrome()
driver.get(f"https://search.shopping.naver.com/search/all?where=all&frm=NVSCTAB&query={find}")

# 반복할 횟수 설정
repeat_limit = 3

while no <= repeat_limit:
    try:
        title = driver.find_element(by='xpath',value = f'//*[@id="content"]/div[1]/div[2]/div/div[{ind}]/div/div/div[2]/div[1]/a').text
        price = driver.find_element(by='xpath',value = f'//*[@id="content"]/div[1]/div[2]/div/div[{ind}]/div/div/div[2]/div[2]/strong/span[1]/span/em').text
        list_1.append([title, price])
        
        time.sleep(random_wait)  # import time
        ind += 1

    except:
        # 다음 지역으로 넘어가기
        try:
            driver.find_element(by='xpath', value = f'//*[@id="content"]/div[1]/div[4]/div/a[{no + 1}]').click()
            no += 1
            ind = 1
            
            time.sleep(random_wait)  # import time

            # 중간 저장
            df = pd.DataFrame(list_1)
            df.to_csv(f"네이버쇼핑 {find}.csv", index = False)
            print(f"{no} 페이지 까지 중간 저장 성공")
            continue
        
        except:
            break

In [ ]:
# 칼럼명 변경 후 저장
df = df.rename(columns = {0 : '상품명', 1 : '가격'})
df.to_csv(f"네이버쇼핑 {find}.csv", index = False)